In [ ]:
# The first thing we need to do is import the required libraries

import csv
import math
import random

In [ ]:
class Decisiond_tree():
    d_tree = {}

    def learn(self, training_data, features, target):
        # def train(training_data, features, target):
      training_data = [item for item in training_data]
      vals = [record[features.index(target)] for record in training_data]
      frequency = {key:0 for key in [item[-1] for item in training_data]}
      idx = features.index(target)
      for tup in training_data:
          frequency[tup[idx]] += 1
      max, major = 0, 0
      for key in frequency.keys():
          if frequency[key]>max:
              max = frequency[key]
              major = key

      if not training_data:
          return major
      elif (len(features) - 1) <= 0:
          return major
      elif vals.count(vals[0]) == len(vals):
          return vals[0]
      else:
          best = features[0]
          minimum_gini = 10000;

          for attr in features:
              new_gini_index = get_gini(features, training_data, attr, target)
              if new_gini_index<minimum_gini:
                  minimum_gini = new_gini_index
                  best = attr
          d_tree = {best:{}}

          idx = features.index(best)
          got_vals = []
          for entry in training_data:
              if entry[idx] not in got_vals:
                  got_vals.append(float(entry[idx]))
          for val in got_vals: #get_values(data, features, best):
              new_data = [[entry[i] for i in range(0, len(entry)) if i != idx] for entry in training_data if entry[idx] == val]
              newAttr = features[:]
              newAttr.remove(best)
              subtree = self.learn(new_data, newAttr, target)
              d_tree[best][val] = subtree
      return d_tree
      
    def classify(self, test_instance, features):
        # Followinf node is to make sure that we create an entity as node
        # the node will store the name of a node in the tree and its children
        class Node():
          def __init__(self, val, dictionary):
              self.value = val
              if (isinstance(dictionary, dict)):
                  self.children = dictionary.keys()
        result = 0 # baseline: always classifies as 0
        # create a copy of d_tree in the varable temp_dict
        temp_dict = self.d_tree.copy()

        # Traverse the tree untill reaching the leaf node
        while(isinstance(temp_dict, dict)):
          # get to the root node of the currect subtree
          root = Node(list(temp_dict.keys())[0], temp_dict[list(temp_dict.keys())[0]])
          # Store the children of the current node in the temp_dict variable
          temp_dict = temp_dict[list(temp_dict.keys())[0]]
          # store the idx of the attribute of the current node and store it in idx variable
          idx = features.index(root.value)
          # Pick the value from the test tuple for the variable location stored in idx variable
          value = test_instance[idx]
          # store the keys of the 
          if value in temp_dict.keys():
            child = Node(value, temp_dict[value])
            result = temp_dict[value]
            temp_dict = temp_dict[value]
          else:
            result = None
            break
        return result

    def gini_target(self, features, data, target_attributes):
        # Set up a dictionary to store the no of occurences of the target attribute
        freq = {}
        # Set the sum of probabilitys
        sum_p2 = 0.0
        i = features.index(target_attributes) - 1
        for entry in data:
            if (entry[i] in freq.keys()):
                freq[entry[i]] += 1.0
            else:
                freq[entry[i]]  = 1.0
        for freq in freq.values():
            sum_p2 += (freq/len(data))**2
        gini = 1 - sum_p2
        return gini

    def get_gini(self, features, data, attr, target_attributes):
        # This function will help to calculate the Gini index of the part of data
        # Using this function we will calculate the Gini index for each split made by each attribute
        var_occur_n = {}

        # Subset entropy variable will contain the sum of entropy of each and every split of the attribute
        gini_sum = 0.0
        # store the idx of the current attribute column
        i = features.index(attr)

        # Iterating over the the data provided we will canculate the frequency of occurence of 
        # the unique elements of the attribute
        for entry in data:
            # for each entry in the data update their frequency
            # If the entry is listed as a key then increment the frequency
            # On the other hand if the entry is not listed then add it as a new element
            if (entry[i] in var_occur_n.keys()):
                var_occur_n[entry[i]] += 1.0
            else:
                var_occur_n[entry[i]]  = 1.0

        # For each key in the dictionary canculate the entropy of corresponding subset of data
        # then add those entropy values in the subset_entropy
        for val in var_occur_n.keys():
            Probability_val        = var_occur_n[val] / sum(var_occur_n.values())
            dataSubset     = [entry for entry in data if entry[i] == val]
            gini_sum += Probability_val * gini_target(features, dataSubset, target_attributes)

        return gini_sum


In [ ]:
def run_decision_d_tree():
    
    # Load data set
    with open("wine-dataset.csv") as f:
      data = [line for line in csv.reader(f, delimiter=",")]
      # for line in csv.reader(f, delimiter=","):
      #   try:
      #       data.append(tuple(float(item) for item in line))
      #   except:
      #       data.append(line)
    # Filtering out the first row because it has the colums names
    # These names are not use to our calculation
    data = data[1:]
    data = [tuple(map(float, rows)) for rows in data]
    print("Number of records: %d" % len(data))

    # Split training/test sets
    # You need to modify the following code for cross validation.
    features = list(range(len(data[0])))
    # target containes the idx of the target variable
    target = features[-1]
    K = 10
    # Initialising the acc variable which will contain the accuracy for each fold
    # We will use the acc list to get the final average accuracy
    acc = []
    for k in range(K):
      # For each iterations of the total 10 folds the data will be shuffled
      random.shuffle(data)
      # Now that shuffled data will will be split into training and test sets
      training_data = [x for i, x in enumerate(data) if i % K != k]
      test_set = [x for i, x in enumerate(data) if i % K == k]

      # Now we will initialize the decision tree object
      d_tree = Decisiond_tree()
      # The object created in above step is now going to be trained with tha data
      d_tree.d_tree = d_tree.learn( training_data, features, target )
      # lets initialize a list named as results this variable will contain the resuts 
      # from all the test cases
      results = []
      # Iterate through all the instances of the test set
      for instance in test_set:
        result = d_tree.classify( instance[:-1] , features)
        if result != None:
            results.append(result == instance[-1])

      # Accuracy
      accuracy = float(results.count(True))/float(len(results))
      acc.append(accuracy)
    # The avg accuracy is stored in avg_acc variable
    avg_acc = sum(acc)/len(acc)
    print("Average accuracy: %.4f" % avg_acc)

    # Writing results to a file (DO NOT CHANGE)
    f = open(myname+"result.txt", "w")
    f.write("accuracy: %.4f" % accuracy)
    f.close()


if __name__ == "__main__":
    run_decision_d_tree()


In [1]:
# Exetremely Fast Decision Tree i.e. Hoeffding Any Time Tree, described in
# "Extremely Fast Decision Tree" (Manapragada, Webb & Salehi, 2018)
#
# this program contains 2 classes: Efdt, EfdtNode
# changed to CART: Gini index
#
# Jamie
# 06/06/2018
#

import time
import numpy as np
import pandas as pd
from multiprocessing import Pool
from itertools import combinations
from sklearn.metrics import accuracy_score


# EFDT node class
class EfdtNode:
    def __init__(self, possible_split_features):
        """
        nijk: statistics of feature i, value j, class
        possible_split_features: features list
        """
        self.parent = None
        self.left_child = None
        self.right_child = None
        self.split_feature = None
        self.split_value = None
        self.split_g = None
        self.new_examples_seen = 0
        self.total_examples_seen = 0
        self.class_frequency = {}
        self.nijk = {i: {} for i in possible_split_features}
        self.possible_split_features = possible_split_features

    def add_children(self, left, right, split_feature, split_value):
        self.left_child = left
        self.right_child = right
        left.parent = self
        right.parent = self
        self.nijk = {i: {} for i in self.nijk.keys()}

        if isinstance(split_value, list):
            left_value = split_value[0]
            right_value = split_value[1]
            if len(left_value) <= 1:
                new_features = [None if f == split_feature else f for f in left.possible_split_features]
                left.possible_split_features = new_features
            if len(right_value) <= 1:
                new_features = [None if f == split_feature else f for f in right.possible_split_features]
                right.possible_split_features = new_features

    def is_leaf(self):
        return self.left_child is None and self.right_child is None

    # update node stats in order to calculate Gini
    def update_stats(self, x, y):
        nijk = self.nijk
        feats = self.possible_split_features
        iterator = [f for f in feats if f is not None]
        for i in iterator:
            value = x[feats.index(i)]

            if value not in nijk[i]:
                nijk[i][value] = {y: 1}
            else:
                try:
                    nijk[i][value][y] += 1
                except KeyError:
                    nijk[i][value][y] = 1

        self.total_examples_seen += 1
        self.new_examples_seen += 1
        class_frequency = self.class_frequency
        try:
            class_frequency[y] += 1
        except KeyError:
            class_frequency[y] = 1

    # the most frequent classification
    def most_frequent(self):
        if self.class_frequency:
            prediction = max(self.class_frequency, key=self.class_frequency.get)
        else:
            # if self.class_frequency dict is empty, go back to parent
            class_frequency = self.parent.class_frequency
            prediction = max(class_frequency, key=class_frequency.get)
        return prediction

    def check_not_splitting(self, class_frequency):
        # compute Gini index for not splitting
        g_d1 = 1
        # g_d2 = 1
        # most = max(class_frequency, key=class_frequency.get)
        n = sum(class_frequency.values())
        for j, k in class_frequency.items():
            g_d1 -= (k/n)**2
        return g_d1

    def node_split(self, split_feature, split_value):
        self.split_feature = split_feature
        self.split_value = split_value
        features = self.possible_split_features

        left = EfdtNode(features)
        right = EfdtNode(features)
        self.add_children(left, right, split_feature, split_value)

    # recursively trace down the tree
    def sort_example(self, x, y, delta, nmin, tau):
        self.update_stats(x, y)
        self.re_evaluate_split(delta, nmin, tau)
        if self.is_leaf():
            self.attempt_split(delta, nmin, tau)
            return
        else:
            left = self.left_child
            right = self.right_child

            index = self.possible_split_features.index(self.split_feature)
            value = x[index]
            split_value = self.split_value

            if isinstance(split_value, list):  # discrete value
                if value in split_value[0]:
                    return left.sort_example(x, y, delta, nmin, tau)
                else:
                    return right.sort_example(x, y, delta, nmin, tau)
            else:  # continuous value
                if value <= split_value:
                    return left.sort_example(x, y, delta, nmin, tau)
                else:
                    return right.sort_example(x, y, delta, nmin, tau)

    def sort_to_predict(self, x):
        if self.is_leaf():
            return self
        else:
            index = self.possible_split_features.index(self.split_feature)
            value = x[index]
            split_value = self.split_value
            if isinstance(split_value, list):  # discrete value
                if value in split_value[0]:
                    return self.left_child.sort_to_predict(x)
                else:
                    return self.right_child.sort_to_predict(x)
            else:  # continuous value
                if value <= split_value:
                    return self.left_child.sort_to_predict(x)
                else:
                    return self.right_child.sort_to_predict(x)

    # test node split, return the split feature
    def attempt_split(self, delta, nmin, tau):
        if self.new_examples_seen < nmin:
            return
        class_frequency = self.class_frequency
        if len(class_frequency) <= 1:
            return

        self.new_examples_seen = 0  # reset
        nijk = self.nijk
        g_Xa = 1  # minimum g

        Xa = ''
        split_value = None
        for feature in self.possible_split_features:
            if feature is not None:
                njk = nijk[feature]
                gini, value = self.gini(njk, class_frequency)
                if gini < g_Xa:
                    g_Xa = gini
                    Xa = feature
                    split_value = value

        epsilon = self.hoeffding_bound(delta)
        g_X0 = self.check_not_splitting(class_frequency)
        g_Xb = g_X0
        if g_Xa < g_X0:
            if g_Xb - g_Xa > epsilon:
                self.split_g = g_Xa  # split on feature Xa
                # print('1 node split')
                self.node_split(Xa, split_value)
            elif g_Xb - g_Xa < epsilon < tau:
                self.split_g = g_Xa  # split on feature Xa
                # print('2 node split')
                self.node_split(Xa, split_value)

    def re_evaluate_split(self, delta, nmin, tau):
        if self.new_examples_seen < nmin or self.is_leaf():  # only re-evaluate non-leaf
            return
        class_frequency = self.class_frequency
        if len(class_frequency) <= 1:
            return

        self.new_examples_seen = 0  # reset
        nijk = self.nijk
        g_Xa = 1
        Xa = ''
        split_value = None
        for feature in self.possible_split_features:
            if feature is not None:
                njk = nijk[feature]
                gini, value = self.gini(njk, class_frequency)
                if gini < g_Xa:
                    g_Xa = gini
                    Xa = feature
                    split_value = value

        epsilon = self.hoeffding_bound(delta)
        g_X0 = self.check_not_splitting(class_frequency)
        split_g = self.split_g  # gini of current split feature

        if g_X0 < g_Xa:  # not split
            print('kill subtree')
            self.kill_subtree()
        if split_g - g_Xa > epsilon or split_g - g_Xa < epsilon < tau:
            if Xa != self.split_feature:
                # print('split on new feature')
                self.split_g = g_Xa  # split on feature Xa
                self.node_split(Xa, split_value)

    def kill_subtree(self):
        if not self.is_leaf():
            self.left_child = None
            self.right_child = None
            self.split_feature = None
            self.split_value = None
            self.split_g = None

    def hoeffding_bound(self, delta):
        n = self.total_examples_seen
        R = np.log(len(self.class_frequency))
        return (R * R * np.log(1/delta) / (2 * n))**0.5

    def gini(self, njk, class_frequency):
        # Gini(D) = 1 - Sum(pi^2)
        # Gini(D, F=f) = |D1|/|D|*Gini(D1) + |D2|/|D|*Gini(D2)
        D = self.total_examples_seen
        m1 = 1  # minimum gini
        # m2 = 1  # second minimum gini
        Xa_value = None
        feature_values = list(njk.keys())  # list() is essential
        if not isinstance(feature_values[0], str):  # numeric feature values
            sort = np.array(sorted(feature_values))
            split = (sort[0:-1] + sort[1:])/2   # vectorized computation, like in R

            D1_class_frequency = {j:0 for j in class_frequency.keys()}
            for index in range(len(split)):
                nk = njk[sort[index]]

                for j in nk:
                    D1_class_frequency[j] += nk[j]
                D1 = sum(D1_class_frequency.values())
                D2 = D - D1
                g_d1 = 1
                g_d2 = 1

                D2_class_frequency = {}
                for key, value in class_frequency.items():
                    if key in D1_class_frequency:
                        D2_class_frequency[key] = value - D1_class_frequency[key]
                    else:
                        D2_class_frequency[key] = value

                for key, v in D1_class_frequency.items():
                    g_d1 -= (v/D1)**2
                for key, v in D2_class_frequency.items():
                    g_d2 -= (v/D2)**2
                g = g_d1*D1/D + g_d2*D2/D
                if g < m1:
                    m1 = g
                    Xa_value = split[index]
                # elif m1 < g < m2:
                    # m2 = g

            return [m1, Xa_value]

        else:  # discrete feature_values
            length = len(njk)
            if length > 9:  # too many discrete feature values, estimate
                for j, k in njk.items():
                    D1 = sum(k.values())
                    D2 = D - D1
                    g_d1 = 1
                    g_d2 = 1

                    D2_class_frequency = {}
                    for key, value in class_frequency.items():
                        if key in k:
                            D2_class_frequency[key] = value - k[key]
                        else:
                            D2_class_frequency[key] = value
                    for key, v in k.items():
                        g_d1 -= (v/D1)**2

                    if D2 != 0:
                        for key, v in D2_class_frequency.items():
                            g_d2 -= (v/D2)**2
                    g = g_d1*D1/D + g_d2*D2/D

                    if g < m1:
                        m1 = g
                        Xa_value = j
                    # elif m1 < g < m2:
                        # m2 = g
                right = list(np.setdiff1d(feature_values, Xa_value))

            else:  # fewer discrete feature values, get combinations
                comb = self.select_combinations(feature_values)
                for i in comb:
                    left = list(i)
                    D1_class_frequency = {key: 0 for key in class_frequency.keys()}
                    D2_class_frequency = {key: 0 for key in class_frequency.keys()}
                    for j,k in njk.items():
                        for key, value in class_frequency.items():
                            if j in left:
                                if key in k:
                                    D1_class_frequency[key] += k[key]
                            else:
                                if key in k:
                                    D2_class_frequency[key] += k[key]
                    g_d1 = 1
                    g_d2 = 1
                    D1 = sum(D1_class_frequency.values())
                    D2 = D - D1
                    for key1, v1 in D1_class_frequency.items():
                        g_d1 -= (v1/D1)**2
                    for key2, v2 in D2_class_frequency.items():
                        g_d2 -= (v2/D2)**2
                    g = g_d1*D1/D + g_d2*D2/D

                    if g < m1:
                        m1 = g
                        Xa_value = left
                    # elif m1 < g < m2:
                        # m2 = g
                right = list(np.setdiff1d(feature_values, Xa_value))
            return [m1, [Xa_value, right]]

    # divide values into two groups, return the combination of left groups
    @staticmethod
    def select_combinations(feature_values):
        combination = []
        e = len(feature_values)
        if e % 2 == 0:
            end = int(e/2)
            for i in range(1, end+1):
                if i == end:
                    cmb = list(combinations(feature_values, i))
                    enough = int(len(cmb)/2)
                    combination.extend(cmb[:enough])
                else:
                    combination.extend(combinations(feature_values, i))
        else:
            end = int((e-1)/2)
            for i in range(1, end+1):
                combination.extend(combinations(feature_values, i))
        return combination


class Efdt:
    def __init__(self, features, delta=0.05, nmin=50, tau=0.1):
        """
        :features: list of data features
        :delta: used to compute hoeffding bound, error rate
        :nmin: to limit the G computations
        :tau: to deal with ties
        """
        self.features = features
        self.delta = delta
        self.nmin = nmin
        self.tau = tau
        self.root = EfdtNode(features)
        self.n_examples_processed = 0

    # find the path of example
    def find_path(self, leaf):
        path = []
        node = leaf
        while node:
            path.append(node)
            node = node.parent
        path.reverse()
        return path

    # update the tree by adding training example
    def update(self, x, y):
        self.n_examples_processed += 1
        self.root.sort_example(x, y, self.delta, self.nmin, self.tau)

    # predict test example's classification
    def predict(self, x_test):
        prediction = []
        if isinstance(x_test, np.ndarray) or isinstance(x_test, list):
            for x in x_test:
                leaf = self.root.sort_to_predict(x)
                prediction.append(leaf.most_frequent())
            return prediction
        else:
            leaf = self.root.sort_to_predict(x_test)
            return leaf.most_frequent()

    def print_tree(self, node):
        if node.is_leaf():
            print('Leaf')
        else:
            print(node.split_feature)
            self.print_tree(node.left_child)
            self.print_tree(node.right_child)


def test_run():
    start_time = time.time()
    # bank.csv whole data size: 4521 # skiprows=1, nrows=n
    df = pd.read_csv('./dataset/bank.csv', header=0, sep=';')
    # df = pd.read_csv('./dataset/default_of_credit_card_clients.csv', skiprows=1, header=0)
    # df = df.drop(df.columns[0], axis=1)
    df = df.sample(frac=1).reset_index(drop=True)  # shuffle data rows
    title = list(df.columns.values)
    features = title[:-1]
    rows = df.shape[0]

    ''' change month string to int '''
    def month_str_to_int(df1):
        import calendar
        d = dict((v.lower(),k) for k,v in enumerate(calendar.month_abbr))
        df1.month = df1.month.map(d)
    month_str_to_int(df)

    # convert df to data examples
    training_size = 4000
    array = df.head(training_size).values
    set1 = array[:1000, :]
    set2 = array[1000:2000, :]
    set3 = array[2000:, :]

    # to simulate continuous training, modify the tree for each training set
    examples = [set1, set2, set3]

    # test set is different from training set
    n_test = 500
    test_set = df.tail(n_test).values
    x_test = test_set[:, :-1]
    y_test = test_set[:, -1]

    # Heoffding bound (epsilon) parameter delta: with 1 - delta probability
    # the true mean is at least r_bar - epsilon
    # Efdt parameter nmin: test split if new sample size > nmin
    # feature_values: unique values in every feature
    # tie breaking: when difference is so small, split when diff_g < epsilon < tau
    tree = Efdt(features, delta=0.01, nmin=100, tau=0.5)
    print('Total data size: ', rows)
    print('Training size: ', training_size)
    print('Test set size: ', n_test)
    n = 0
    for training_set in examples:
        n += len(training_set)
        x_train = training_set[:, :-1]
        y_train = training_set[:, -1]
        for x, y in zip(x_train, y_train):
            tree.update(x, y)
        y_pred = tree.predict(x_test)
        print('Training samples:', n, end=', ')
        print('ACCURACY: %.4f' % accuracy_score(y_test, y_pred))

    print("--- Running time: %.6f seconds ---" % (time.time() - start_time))


if __name__ == "__main__":
    test_run()

Total data size:  4521
Training size:  4000
Test set size:  500
Training samples: 1000, ACCURACY: 0.8960
Training samples: 2000, ACCURACY: 0.9020
Training samples: 4000, ACCURACY: 0.9020
--- Running time: 0.401111 seconds ---


In [2]:
# Very Fast Decision Tree i.e. Hoeffding Tree, described in
# "Mining High-Speed Data Streams" (Domingos & Hulten, 2000)
#
# this program contains 2 classes: vfdt, vfdt_node
# test in command line window: python3 vfdt.py
# changed to CART: Gini index
#
# Jamie
# 02/06/2018
# ver 0.03


import numpy as np
import pandas as pd
import math
import time
from itertools import combinations


# VFDT node class
class vfdt_node:
    # parameter nijk: statistics of feature i, value j, class k
    def __init__(self, possible_split_features):
        self.parent = None
        self.left_child = None
        self.right_child = None
        self.split_feature = None
        self.split_value = None
        self.new_examples_seen = 0
        self.total_examples_seen = 0
        self.class_frequency = {}
        self.nijk = {i:{} for i in possible_split_features}
        self.possible_split_features = possible_split_features

    def add_children(self, split_feature, left, right):
        self.split_feature = split_feature
        self.left_child = left
        self.right_child = right
        left.parent = self
        right.parent = self
        self.nijk.clear()  # reset stats

    # recursively trace down the tree to distribute data examples to corresponding leaves
    def sort_example(self, example):
        if (not self.is_leaf()):
            index = self.possible_split_features.index(self.split_feature)
            value = example[:-1][index]

            try:  # continous value
                if value <= self.split_value:
                    return self.left_child.sort_example(example)
                else:
                    return self.right_child.sort_example(example)
            except TypeError:  # discrete value
                if value in self.split_value[0]:
                    return self.left_child.sort_example(example)
                else:
                    return self.right_child.sort_example(example)
        else:
            return(self)

    def is_leaf(self):
        return(self.left_child == None and self.right_child == None)

    # the most frequent classification
    def most_frequent(self):
        if (self.class_frequency):
            prediction = max(self.class_frequency, key=self.class_frequency.get)
        else:
            # if self.class_frequency dict is empty, go back to parent
            class_frequency = self.parent.class_frequency
            prediction = max(class_frequency, key=class_frequency.get)
        return(prediction)

    # upadate leaf stats in order to calculate infomation gain
    def update_stats(self, example):
        label = example[-1]
        feats = self.possible_split_features
        for i in feats:
            if (i != None):
                value = example[:-1][feats.index(i)]
                if (value not in self.nijk[i]):
                    d = {label : 1}
                    self.nijk[i][value] = d
                else:
                    if (label not in self.nijk[i][value]):
                        self.nijk[i][value][label] = 1
                    else:
                        self.nijk[i][value][label] += 1
        self.total_examples_seen += 1
        self.new_examples_seen += 1
        if (label not in self.class_frequency):
            self.class_frequency[label] = 1
        else:
            self.class_frequency[label] += 1

    def check_not_splitting(self):
        # compute Gini index for not splitting
        X0 = 1
        class_frequency = self.class_frequency
        n = sum(class_frequency.values())
        for j, k in class_frequency.items():
            X0 -= (k/n)**2
        return X0

    # use hoeffding tree model to test node split, return the split feature
    def splittable(self, delta, nmin, tau):
        if(self.new_examples_seen < nmin):
            return(None)
        else:
            self.new_examples_seen = 0  # reset
        nijk = self.nijk
        min = 1
        second_min = 1
        Xa = ''
        split_value = None
        for feature in self.possible_split_features:
            if (len(nijk[feature]) != 1):
                gini, value = self.Gini(feature)
                if (gini < min):
                    min = gini
                    Xa = feature
                    split_value = value
                elif (min < gini < second_min):
                    second_min = gini
            else:
                return None

        sigma = self.hoeffding_bound(delta)
        X0 = self.check_not_splitting()
        if min < X0:
            if (second_min - min > sigma):
                return [Xa, split_value]
            elif (sigma < tau and second_min - min < tau):
                return [Xa, split_value]
            else:
                return None
        else:
            return None

    def hoeffding_bound(self, delta):
        n = self.total_examples_seen
        R = np.log(len(self.class_frequency))
        return (R * R * np.log(1/delta) / (2 * n))**0.5

    def Gini(self, feature):
        '''
        Gini(D) = 1 - Sum(pi^2)
        Gini(D, F=f) = |D1|/|D|*Gini(D1) + |D2|/|D|*Gini(D2)
        '''
        njk = self.nijk[feature]
        D = self.total_examples_seen
        class_frequency = self.class_frequency

        m1 = 1  # minimum gini
        m2 = 1  # second minimum gini
        Xa_value = None
        test = next(iter(njk))  # test j value
        # if not isinstance(test, np.object):
        try:  # continous feature values
            test += 0
            sort = sorted([j for j in njk.keys()])
            split = []
            for i in range(1, len(sort)):
                temp = (sort[i-1] + sort[i])/2
                split.append(temp)

            D1 = 0
            D1_class_frequency = {j:0 for j in class_frequency.keys()}
            for index in range(len(split)):
                nk = njk[sort[index]]

                for j in nk:
                    D1_class_frequency[j] += nk[j]
                D1 += sum(nk.values())
                D2 = D - D1
                g_d1 = 1
                g_d2 = 1

                D2_class_frequency = {}
                for key, value in class_frequency.items():
                    if key in D1_class_frequency:
                        D2_class_frequency[key] = value - D1_class_frequency[key]
                    else:
                        D2_class_frequency[key] = value

                for key, v in D1_class_frequency.items():
                    g_d1 -= (v/float(D1))**2
                for key, v in D2_class_frequency.items():
                    g_d2 -= (v/float(D2))**2
                g = g_d1*D1/D + g_d2*D2/D
                if g < m1:
                    m1 = g
                    Xa_value = split[index]
                elif m1 < g < m2:
                    m2 = g
            return [m1, Xa_value]

        # discrete feature_values
        except TypeError:
            length = len(njk)
            feature_values = list(njk.keys())
            right = None
            if length > 10:  # too many discrete feature values
                for j, k in njk.items():
                    D1 = sum(k.values())
                    D2 = D - D1
                    g_d1 = 1
                    g_d2 = 1

                    D2_class_frequency = {}
                    for key, value in class_frequency.items():
                        if key in k:
                            D2_class_frequency[key] = value - k[key]
                        else:
                            D2_class_frequency[key] = value

                    for key, v in k.items():
                        g_d1 -= (v/D1)**2

                    if D2 != 0:
                        for key, v in D2_class_frequency.items():
                            g_d2 -= (v/D2)**2
                    g = g_d1*D1/D + g_d2*D2/D
                    if g < m1:
                        m1 = g
                        Xa_value = j
                    elif m1 < g < m2:
                        m2 = g
                right = list(np.setdiff1d(feature_values, Xa_value))

            else:  # fewer discrete feature values
                comb = self.select_combinations(feature_values)
                for i in comb:
                    left = list(i)
                    D1 = 0
                    D2 = 0
                    D1_class_frequency = {key:0 for key in class_frequency.keys()}
                    D2_class_frequency = {key:0 for key in class_frequency.keys()}
                    for j,k in njk.items():
                        for key, value in class_frequency.items():
                            if j in left:
                                if key in k:
                                    D1 += sum(k.values())
                                    D1_class_frequency[key] += k[key]
                            else:
                                if key in k:
                                    D2 += sum(k.values())
                                    D2_class_frequency[key] += k[key]
                    g_d1 = 1
                    g_d2 = 1
                    for key, v in D1_class_frequency.items():
                        g_d1 -= (v/D1)**2
                    for key, v in D2_class_frequency.items():
                        g_d2 -= (v/D1)**2
                    g = g_d1*D1/D + g_d2*D2/D
                    if g < m1:
                        m1 = g
                        Xa_value = left
                    elif m1 < g < m2:
                        m2 = g
                right = list(np.setdiff1d(feature_values, Xa_value))
            return [m1, [Xa_value, right]]

    def select_combinations(self, feature_values):
        combination = []
        e = len(feature_values)
        if e % 2 == 0:
            end = int(e/2)
            for i in range(1, end+1):
                if i == end:
                    cmb = list(combinations(feature_values, i))
                    enough = int(len(cmb)/2)
                    combination.extend(cmb[:enough])
                else:
                    combination.extend(combinations(feature_values, i))
        else:
            end = int((e-1)/2)
            for i in range(1, end+1):
                combination.extend(combinations(feature_values, i))

        return combination

# very fast decision tree class, i.e. hoeffding tree
class vfdt:
    # parameters
    # feature_values  # number of values of each feature # dict
    # feature_values = {feature:[unique values list]}
    # delta   # used for hoeffding bound
    # tau  # used to deal with ties
    # nmin  # used to limit the G computations

    def __init__(self, feature_values, delta=0.05, nmin=50, tau=0.1):
        self.feature_values = feature_values
        self.delta = delta
        self.nmin = nmin
        self.tau = tau
        features = list(feature_values.keys())
        self.root = vfdt_node(features)
        self.n_examples_processed = 0

    # update the tree by adding training example
    def update(self, example):
        self.n_examples_processed += 1
        node = self.root.sort_example(example)
        node.update_stats(example)

        result = node.splittable(self.delta, self.nmin, self.tau)
        if result != None:
            feature = result[0]
            value = result[1]
            #print('SPLIT')
            #print(feature, value)
            self.node_split(node, feature)
            node.split_value = value

    # split node, produce children
    def node_split(self, node, split_feature):
        features = node.possible_split_features
        # replace deleted split feature with None
        # new_features = [None if f == split_feature else f for f in features]
        left = vfdt_node(features)
        right = vfdt_node(features)
        node.add_children(split_feature, left, right)

    # predict test example's classification
    def predict(self, example):
        leaf = self.root.sort_example(example)
        prediction = leaf.most_frequent()
        return(prediction)

    # accuracy of a test set
    def accuracy(self, examples):
        correct = 0
        for ex in examples:
            if (self.predict(ex) == ex[-1]):
                correct +=1
        return(float(correct) / len(examples))

    def print_tree(self, node):
        if node.is_leaf():
            print('Leaf')
        else:
            print(node.split_feature)
            self.print_tree(node.left_child)
            self.print_tree(node.right_child)

def test_run():
    start_time = time.time()

    # bank.csv whole data size: 4521
    # if more than 4521, it revert back to 4521
    rows = 4521
    # n_training = int(0.8 * rows)
    # read_csv has parameter nrows=n that read the first n rows
    '''skiprows=1, index_col=0,'''
    df = pd.read_csv('./dataset/bank.csv', nrows=rows, header=0, sep=';')
    title = list(df.columns.values)
    features = title[:-1]

    ''' change month string to int '''
    import calendar
    d = dict((v.lower(),k) for k,v in enumerate(calendar.month_abbr))
    df.month = df.month.map(d)
    #print(df.head(10))

    # unique values for each feature to use in VFDT
    feature_values = {f:None for f in features}
    for f in features:
        feature_values[f] = df[f].unique()

    # convert df to data examples
    array = df.head(4000).values
    set1 = []
    set2 = []
    set3 = []
    possible_split_features = title[:-1]
    count = 0
    for i in range(len(array)):
        count += 1
        if (count <= 1000):
            set1.append(array[i])
        elif (count > 1000 and count <= 2000):
            set2.append(array[i])
        else:
            set3.append(array[i])
    # to simulate continous training, modify the tree for each training set
    examples = [set1, set2, set3]

    # test set is different from training set
    n_test = 500
    test_set = df.tail(n_test).values
    test = []
    for i in range(len(test_set)):
        test.append(test_set[i])

    # heoffding bound parameter delta: with 1 - delta probability
    # the true mean is at least r - gamma
    # vfdt parameter nmin: test split if new sample size > nmin

    tree = vfdt(feature_values, delta=0.05, nmin=100, tau=0.05)
    print('Total data size: ', rows)
    print('Test set (tail): ', len(test_set))
    n = 0
    for training_set in examples:
        n += len(training_set)
        for ex in training_set:
            tree.update(ex)
        print('Training set:', n, end=', ')
        print('ACCURACY: %.4f' % tree.accuracy(test))

    tree.print_tree(tree.root)
    print("--- Running time: %.6f seconds ---" % (time.time() - start_time))

if __name__ == "__main__":
    test_run()

Total data size:  4521
Test set (tail):  500
Training set: 1000, ACCURACY: 0.8840
Training set: 2000, ACCURACY: 0.8840
Training set: 4000, ACCURACY: 0.8860
poutcome
Leaf
poutcome
Leaf
contact
Leaf
education
Leaf
marital
Leaf
education
Leaf
duration
duration
duration
contact
Leaf
Leaf
poutcome
Leaf
Leaf
Leaf
Leaf
--- Running time: 0.088289 seconds ---
